### 제네레이터 (Generator, 발생자)


In [2]:
# 기존의 함수 호출 방식은 함수가 호출될 때 인수들의 내부 변수들이
# 새로운 영역(스택)에 만들어지고 반환시 메모리에서 소멸된다
def f(a,b):
    c = a * b
    d = a + b
    return c,d

print(f(10,20))
print(f(1,2))

(200, 30)
(2, 3)


In [5]:
# 제네레이터는 중단된 시점부터 재 실행이 가능한 함수 
# return 대신 키워드 yield 를 사용한다. 
def gen_func():
    print('1st step')
    yield 1
    print('2nd step')
    yield 2
    print('3nd step')
    yield 3
    
g = gen_func() # generator 객체
print(type(g))

a = next(g)  # next로 호출 # 최초 호출
print(a)

<class 'generator'>
1st step
1


In [6]:
b = next(g)
print(b)

2nd step
2


In [7]:
c = next(g)
print(c)

3nd step
3


In [8]:
d = next(g)
print(d)  # 더 호출하면 error

StopIteration: 

In [9]:
# generator 를 for 문으로 호출
def generator_ints(N):
    for k in range(N):
        yield k
        
for k in generator_ints(10):
    print(k,end=' ')
    
print(type(generator_ints(10))) #제네레이터는 반복자 객체이다. 

0 1 2 3 4 5 6 7 8 9 <class 'generator'>


### 코루틴 (Coroutine)
: 함수 실행에 있어서 어떤 위치에서 중단과 실행이 가능한 다중 진입점이 있는 일반화된 함수

In [10]:
def echo():
    print('echo routine')
    while True:
        msg = (yield)
        print('echo:',msg)

In [11]:
e = echo()
print(type(e))
next(e)

<class 'generator'>
echo routine


In [12]:
next(e)

echo: None


In [13]:
# 코루틴은 send를 보냄 
e.send('Hello~')

echo: Hello~


In [14]:
e.send('Bye~')
e.close()

echo: Bye~


In [15]:
# 양방향 값 전송 코루틴
def accumulate(value= 0):
    acc = value
    while True:
        value = (yield acc, value)
        acc += value
        
acc = accumulate(1)
print(type(acc))
next(acc)

<class 'generator'>


(1, 1)

In [16]:
acc.send(2) # 2를 value 에 전달해주고 다음 yield 문까지 진행, (3,2)를 반환받는다.

(3, 2)

In [17]:
acc.send(3) # 3를 valude 에 전달해주고 (6,3)를 반환

(6, 3)

## (3) 미니 챗봇 구현

In [19]:
import re
r = "(hi|hello|hey)[ ]*([a-z]*)" #빈칸이 없거나 0번이상. 
re.match(r,'Hello Rosa',re.I) # re.i -> 대소문자 상관 안함.

<re.Match object; span=(0, 10), match='Hello Rosa'>

In [21]:
re.match(r,"hi ho, my name is sunyoung")

<re.Match object; span=(0, 5), match='hi ho'>

In [23]:
re.match(r,"hey, what's up", re.I) #쉼표 때문에

<re.Match object; span=(0, 3), match='hey'>

In [26]:
# 정규 표현식을 더 확장
r = r"[^a-z]*([y]o|[h]?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r,re.I)
# r모드 , 처음에 문자가 아닌것이 0 개 이상으로 인사(1 hi)<- 숫자가 나와도 문제가 없게 
# yo, oh, ?ello - > 안나오거나 0번. ello 도 가능, 근데 2번하면 틀림 ?는 0번,1번만 의미
# ok, hey
# good 은 good morning 등으로 한칸이 띄어지는 것으로 봄 ?<- good 어쩌고가 0번 또는 1번
# gin'-> moring 이 나와도 되고 0에서 3번, mor 이런식으로도 써도 가능  morn 도 가능
# [\ㄴ,:;] -> hello; rosa가능, hellorosa이거는 안됨.
# 20글자안에까지 이름 가능.

In [27]:
re_greeting.match("Hello Rosa")

<re.Match object; span=(0, 10), match='Hello Rosa'>

In [28]:
re_greeting.match("Good Morning Rosa")

<re.Match object; span=(0, 17), match='Good Morning Rosa'>

In [29]:
re_greeting.match("Good Marning Rosa")

In [30]:
re_greeting.match("Good Evening Rosa")

<re.Match object; span=(0, 17), match='Good Evening Rosa'>

In [33]:
re_greeting.match("Good Evening Rosa Parks").group(1)

'Good Evening'

In [34]:
re_greeting.match("Good Evening Rosa Parks").groups()

('Good Evening', 'Good ', 'Evening', 'Rosa')

In [35]:
re_greeting.match("Good morn'n Rosa")

<re.Match object; span=(0, 16), match="Good morn'n Rosa">

In [36]:
re_greeting.match("yo Rosa")

<re.Match object; span=(0, 7), match='yo Rosa'>

In [37]:
re_greeting.match("ello Rosa")

<re.Match object; span=(0, 9), match='ello Rosa'>

In [38]:
re_greeting.match("hellohello Rosa")

In [40]:
# Chatbot
r = r"[^a-z]*([y]o|[h]?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r,re.I)

my_names = set(['rosa','rose','chatty','chatbot','bot','chatterbot','baby']) # 좋은 단어
curt_names = set(['hal','you','u','guy','boy']) #무뚝뚝한 단어
greeter_name ='선영' # 대화 상대자 (사람)

match = re_greeting.match(input()) # 'Hello Rosa'

if match :
    at_name = match.groups()[-1] # 4개 그룹중 마지막 그룹
    if at_name in curt_names:
        print('Um... Have a good time.')
    elif at_name.lower() in my_names:
        print('Hi {}, How are you? Glad to see you.'.format(greeter_name))

hello guy
Um... Have a good time.


## Chat test

In [41]:
# evil_names 목록을 넣어 대화에 사용하면 화를 내는 코드를 간단히 추가하고 동작을 확인 해보세요
# 영어 대답 아래에 동일한 한국어 대답도 출력하게 해보세요
# while True를 사용하여 match가 없을 때 종료하도록 수정하세요
# evil_names : set(['idiot','fool','ass','sob'])
# 대화가 시작되었습니다!
# ----------------------------------------------------------------------
# 길동 : Hello Rosa
# 컴퓨터 : Hi 길동, How are you? Glad to see you.
# 컴퓨터 : 길동씨 안녕하세요,만나서 반갑습니다
# ----------------------------------------------------------------------
# 길동 : Hi you
# 컴퓨터 : Um.. Have a good time.
# 컴퓨터 : 음.. 즐거운 시간 되세요.
# ----------------------------------------------------------------------
# 길동 : Hey idiot
# 컴퓨터 : Dear 길동 please, don't talk to me like that : idiot. 
# 컴퓨터 : 길동아! 제발 그런말 쓰지마! : idiot. 
# ----------------------------------------------------------------------
# 길동 : Bye
# 대화가 종료되었습니다!

In [67]:
# Chatbot
r = r"[^a-z]*([y]o|[h]?ello|ok|hi|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r,re.I)

my_names = set(['rosa','rose','chatty','chatbot','bot','chatterbot','baby']) # 좋은 단어
curt_names = set(['hal','you','u','guy','boy']) #무뚝뚝한 단어
evil_names = set(['idiot','fool','ass','sob'])
greeter_name ='선영' # 대화 상대자 (사람)

match = re_greeting.match(input()) # 'Hello Rosa'

while True:
    if match :
        at_name = match.groups()[-1] # 4개 그룹중 마지막 그룹
        if at_name in curt_names:
            print('Um... Have a good time.')
            print("음... 즐거운 시간 되세요")
        elif at_name in my_names:
            print('Hi {}, How are you? Glad to see you.'.format(greeter_name))
            print('{}씨 안녕하세요. 만나서 반갑습니다.'.format(greeter_name))
        elif at_name in evil_names:
            print("Dear {0} please, dont't talk to me like that : {1}".format(greeter_name, at_name))
            print("{0}아! 제발 그런말 쓰지마 : {1}".format(greeter_name, at_name))
        break
    else:
        print("대화가 종료되었습니다.")
        break
    
   
    

hey idiot
Dear 선영 please, dont't talk to me like that : idiot
선영아! 제발 그런말 쓰지마 : idiot


# re.compile

In [71]:
import re
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Seoul_Metropolitan_Subway"
resp = requests.get(url)
html_src = resp.text
soup = BeautifulSoup(html_src,'html.parser')

links = soup.find_all("a")
print('하이퍼링크의 갯수:', len(links))
print('첫 3개의 원소:',links[:3])
print('\n')
# /wiki/ 문자열이 포함된 링크 
wiki_links = soup.find_all(name='a',href=re.compile("/wiki/"),limit=3)
print(wiki_links)

하이퍼링크의 갯수: 963
첫 3개의 원소: [<a id="top"></a>, <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>, <a class="mw-jump-link" href="#searchInput">Jump to search</a>]


[<a class="image" href="/wiki/File:South_Korea_subway_logo.svg"><img alt="South Korea subway logo.svg" data-file-height="450" data-file-width="450" decoding="async" height="75" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/12/South_Korea_subway_logo.svg/75px-South_Korea_subway_logo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/12/South_Korea_subway_logo.svg/113px-South_Korea_subway_logo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/1/12/South_Korea_subway_logo.svg/150px-South_Korea_subway_logo.svg.png 2x" width="75"/></a>, <a class="image" href="/wiki/File:Seoul-Metro-2004-20070722.jpg"><img alt="Seoul-Metro-2004-20070722.jpg" data-file-height="2100" data-file-width="2800" decoding="async" height="225" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/29/Seoul-Metro-20

In [74]:
# 콜론이 포함된 링크 제외
wiki_links = soup.find_all(name='a',href=re.compile("^/wiki/((?!:).)*$"),limit=3)
print('콜론이 포함된 링크가 제외된 하이퍼링크 :',wiki_links)

# ^ 시작은 WIKI로 시작, 콜론이 없는거로 모든게 나오고, $ 끝
# # ?! : 부정형 전방탐색
# https://blog.hexabrain.net/205


콜론이 포함된 링크가 제외된 하이퍼링크 : [<a href="/wiki/Government_of_South_Korea" title="Government of South Korea">Government of South Korea</a>, <a href="/wiki/Seoul_Metropolitan_Government" title="Seoul Metropolitan Government">Seoul Metropolitan Government</a>, <a href="/wiki/Incheon" title="Incheon">Incheon Metropolitan City</a>]


In [75]:
# 정규 표현식 객체를 건네면,  match() 메쏘드를 사용하여 그 정규 표현식에 맞게 여과한다. 

# 다음 코드는 이름이 “b”로 시작하는 태그를 모두 찾는다; 이 경우, <body> 태그와 <b> 태그를 찾을 것이다:

import re
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)
# body
# b

#다음 코드는 이름에 ‘t’가 포함된 태그를 모두 찾는다:

for tag in soup.find_all(re.compile("t")):
    print(tag.name)
# html
# title


body
br
br
br
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
bdi
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
br
br
html
meta
title
script
script
script
meta
meta
meta
meta
meta
meta
table
caption
tbody
tr
td
tr
td
tr
th
tr
th
td
tr
th
td
tr
th
td
tr
th
td
tr
th
td
tr
th
td
tr
th
td
tr
th
tr
th
td
tr
th
td
tr
th
tr
th
td
tr
th
td
table
tbody
tr
td
tr
th
td
tr
th
td
tr
th
td
tr
th
td
input
table
tbody
tr
th
th
th
th
th
th
th
th
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td
td
td
td
td
td
td
td
tr
td
td


In [76]:
# bs4 라이브러리 안에도 정규표현식이 사용되고 있다
# C:\ProgramData\Anaconda3\Lib\site-packages\bs4\_htmlparser.py
# XXX This code can be removed once most Python 3 users are on 3.2.3.
# if major == 3 and minor == 2 and not CONSTRUCTOR_TAKES_STRICT:
#     import re
#     attrfind_tolerant = re.compile(
#         r'\s*((?<=[\'"\s])[^\s/>][^\s/=>]*)(\s*=+\s*'
#시작이 0개 이상 그룹으로 묶고, ?<= 는, 긍정 탐색이 있고, [] 이 안에 글자들이 하나 이상 있고 []이안에것들이 아닌것이 나오고 등등...
#         r'(\'[^\']*\'|"[^"]*"|(?![\'"])[^>\s]*))?')
#     HTMLParserTreeBuilder.attrfind_tolerant = attrfind_tolerant

#     locatestarttagend = re.compile(r"""
#   <[a-zA-Z][-.a-zA-Z0-9:_]*          # tag name
#   (?:\s+                             # whitespace before attribute name
#     (?:[a-zA-Z_][-.:a-zA-Z0-9_]*     # attribute name
#       (?:\s*=\s*                     # value indicator
#         (?:'[^']*'                   # LITA-enclosed value
#           |\"[^\"]*\"                # LIT-enclosed value
#           |[^'\">\s]+                # bare value
#          )
#        )?
#      )
#    )*
#   \s*                                # trailing whitespace


In [77]:
# soup.find_all(
#     name=None,
#     attrs={},
#     recursive=True,
#     text=None,
#     limit=None,
#     **kwargs,
# )

# **kwargs : 키워드 인수
#  dict형식
def func(width,height,**kw):
    print(width,height)
    print(kw,type(kw)) 
    
func(width=10,height=20,depth=5,dimension=7,a =10,href='/wiki/')  #kw 는 depth 이후부터
func(1,2,depth=3,dimension=3) 


10 20
{'depth': 5, 'dimension': 7, 'a': 10, 'href': '/wiki/'} <class 'dict'>
1 2
{'depth': 3, 'dimension': 3} <class 'dict'>
